## Vertex AI Speech-to-Text pretrained model
This notebook demonstrate how you can convert speech audio file to text.

In [1]:
import base64
from google.cloud import speech_v1
import google.auth
from google.auth.transport import requests
from google.api_core.client_options import ClientOptions
import requests as reqs

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "<your service account key file>"
os.environ["GRPC_DEFAULT_SSL_ROOTS_FILE_PATH"] = "<your trusted bundle certs>"

In [ ]:
audience = "https://<vertex ai speech-to-text endpoint>:443"  #e.g. speech.org-1.acme.com
api_endpoint = "<vertex ai speech-to-text endpoint>"

In [4]:
# Get your credentials
def get_credentials():
    creds = None
    try:
        creds, project_id = google.auth.default()
        creds = creds.with_gdch_audience(audience)
        sesh = reqs.Session()
        sesh.verify = False
        creds._ca_cert_path = False
        req = requests.Request(session=sesh)
        creds.refresh(req)
        print("Got token: ")
        print(creds.token)
    except Exception as e:
        print("Caught exception" + str(e))
        raise e
    return creds

In [5]:
def stt_client(creds):
    opts = ClientOptions(api_endpoint=api_endpoint)
    return speech_v1.SpeechClient(credentials=creds, client_options=opts)

In [6]:
def stt(language_code, sample_rate, audio_channel, data):
    creds = get_credentials()
    tc = stt_client(creds)
    content=data
    audio = speech_v1.RecognitionAudio()
    audio.content = base64.standard_b64decode(content)
    config = speech_v1.RecognitionConfig()
    config.encoding= speech_v1.RecognitionConfig.AudioEncoding.LINEAR16
    config.sample_rate_hertz=sample_rate
    config.language_code=language_code
    config.audio_channel_count=audio_channel
    resp = tc.recognize(config=config, audio=audio)
    print("\nResponse: \n",resp)

In [ ]:
stt("en-US", 16000, 1, "UklGRjy2AABXQVZFZm10IBAAAAABAAEAgD4AAAB9AAACABAAZGF0YRi2AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAFAAUABQACAAUACQAFAAwADAAMAAwAEAASABAADAAMABIAEgAQABAADAAVABUAFwAgABwAJQApACMAIAAjACMAIAAjACAAHAAgACAAJQAcAB4AEgAJAAwAHAASABcAFQAcABUAEgAVAA4AEAAMAA4ACQAVABAAEgAOABUAGQAOAAAACQAMABAADgAHAAkABQAJAAAA")